## Compute the emittance growth for white noise case including the contribution of the tune distribution 

The formulas are based on Eq. (20) and (21) from PhysRevSTAB.18.101001

We cosnider:
- no dependence on tune distribution
- overlapse only with the firt betatron sideband

Note: In (20), the noise PSD is sampled twice at +/- the betatron tune. Similarly, in (22), the noise PSD is sampled four times at the two synchrotron sidebands of each of the two betatron sidebands. So, you need a factor of 2 in your expression for phase noise and a factor of 4 for that of amplitude noise (assuming of course that the noise is the same at the those sidebands). by Themis

In [7]:
import scipy.integrate as integrate
import numpy as np
import pickle

In [17]:
def emit_growth_phase_noise(betay, Vcc, frev, Eb, CDeltaPhi, PSD_phi, rho_b): 
    
    ey_rate = betay*(Vcc*frev/(2*Eb))**2*CDeltaPhi*PSD_phi
    
    result = integrate.quad(lambda vb: rho_b(vb), -np.inf, np.inf)
    
    return 2*result[0]*ey_rate

In [9]:
def emit_growth_amplitude_noise(betay, Vcc, frev, Eb, CDeltaA, PSD_A, rho_b):
    
    ey_rate = betay*(Vcc*frev/(2*Eb))**2*CDeltaA*PSD_A
    
    result = integrate.quad(lambda vb: rho_b(vb), -np.inf, np.inf)
    
    
    return 4*result[0]*ey_rate

### SPS parameters

In [10]:
betay = 76 # m
# Beam parameters
Vcc = 1e6# V
frev = 43.45e3 # Hz
Eb = 270e9 # eV
CDeltaPhi = 0.52 # approx
scale_factor = Eb/Vcc # scaling factor to rms noise, σ
PSD_phi = 2.3e-21*(scale_factor)**2 # rad^2/Hz
print('PSD phi = {} rad^2/Hz'.format(PSD_phi))
beta_0 = 0.999999
gamma_0 = 287.7

PSD phi = 1.6767e-10 rad^2/Hz


### Function for tune distribution

In [11]:
def tune_distribution(x, mu, b):
    if x >= 0:
        return (1/b) * np.exp(-(x-mu)/b)
    if x < 0:
        return 0

In [12]:
rho_b = lambda vb : tune_distribution(vb, 0.18, 1/0.00146)

### Call the function computing the emittance

In [16]:
ey_geom_rate_phi = emit_growth_phase_noise(betay, Vcc, frev, Eb, CDeltaPhi, PSD_phi, rho_b)
ey_norm_rate_phi = ey_geom_rate_phi*beta_0*gamma_0

1.0002628345349438


In [14]:
print('Phase noise with PSD={} rad^2/Hz: dey/dt normalised = {} nm/s'.format(PSD_phi, ey_norm_rate_phi*1e9))

Phase noise with PSD=1.6767e-10 rad^2/Hz: dey/dt normalised = 24.6915241092181 nm/s
